# Predicción de Calidad de Café

**Objetivo**  
Predecir la calidad del café basados en el Puntaje de Taza.

---

### 1. Análisis de datos y selección de variables de entrada (X)
- Cargar y explorar el conjunto de datos.  
- Definir las variables predictoras (**X**), cuidando que no se filtre información del target.

### 2. Preprocesamiento
- **X** (features):
  - Imputación de valores faltantes  
  - Normalización / estandarización  
  - Codificación de variables categóricas  
- **Y** (Puntaje de Taza):
  - Revisión de distribución  
  - Transformaciones si aplica (e.g., escalado, discretización)

### 3. Entrenamiento de modelos de regresión
- Modelo 1: _Linear Regression_ (u otro algoritmo lineal)  
- Modelo 2: _Random Forest Regressor_ (u otro algoritmo no lineal)  

### 4. Explicabilidad
- Aplicar técnicas de interpretabilidad:
  - LIME  
  - SHAP  
  - Importancia de características (Feature Importance)

### 5. Reporte de resultados
- Métricas de evaluación: RMSE, MAE, R²  
- Comparación de desempeño entre modelos  
- Visualizaciones e interpretaciones  


In [1]:
#Primero importamos todas las librerias que vamos a utilizar
import os 
import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#Importamos las librerias de sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


### 1.1 Análisis de datos
- Cargar y explorar el conjunto de datos.  

In [ ]:
#Primero Cargamos los datos almacenados en los archivos .xlsx

dict_Calidad = pd.read_excel('CC_FT_17_Calidad.xlsx', 
                             sheet_name=['CONTROL CALIDAD CAFE TRILLADO J','Sheet2'],
                             skiprows=5)

#Accedemos a las hojas específicas del diccionario
#Quitamos las columnas que no son necesarias para el análisis, por ejemplo verificación física del café tostado tiene 
#Unicamente un valor y no aporta información relevante para el análisis

df_Calidad = dict_Calidad['CONTROL CALIDAD CAFE TRILLADO J']

#Las columnas sin nombre en este caso solo tienen un valor: 'C', por lo cual procedemos a eliminarlas.
df_Calidad = df_Calidad.loc[:, ~df_Calidad.columns.str.startswith('Unnamed')]

#quita espacios al principio y al final de cada nombre
df_Calidad.columns = df_Calidad.columns.str.strip()

df_Calidad = df_Calidad.drop(columns=['FECHA','VERIFICACIÓN FISICA CAFÉ TOSTADO', 'LIBERACIÓN DE LOTE', 'RESPONSABLE'])

#Por ultimo eliminamos las filas 0 y 1 que no aportan información relevante y las ultimas filas.

df_Calidad = df_Calidad.drop(index=[0,1])
df_Calidad = df_Calidad.iloc[:-17]


# Realizamos una visualización rápida de los datos en cada hoja

display(df_Calidad.head())
display(df_Calidad.tail())


,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
2,01-190722,Madre Laura,765.00,10.9,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
3,09-190722,Tabi Natural,204.00,10.2,14,"Frutas maduras, nibs de cacao, acidez brillant...",85
4,10-190722,Don Mario,165.00,10.7,14,"Panela, durazno, miel, acidez brillante citric...",84.5
5,07-19-07-22,Don Felix,0.45,10.5,14,"Moras maduras, chocolate negro, acidez media c...",84.5
6,01-291022,Madre Laura,105.00,10.7,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84


,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
73,22-170624,El Ocaso - Caturron,48.0,10.3,15,"Chocolate dulce, nuez moscada, acidez jugosa, ...",87
74,01-030724,Madre Laura,250.0,10.4,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
75,23-150724,Esteban Robledo,60.0,10.6,14,"Panela, almendras, acidez citrica jugosa como ...",85
76,02-150724,Madre Laura Natural,175.0,10.8,14,"Cacao, frutos rojos, cuerpo cremoso, acidez br...",85
77,17-270724,Familia Bedoya Castaño,35.0,10.8,14,"Melao de panela, arandanos, cuerpo cremoso, ac...",87


In [39]:
#Ahora hacemos un tratamiento similar con la segunda hoja del archivo Excel
df_Calidad2 = dict_Calidad['Sheet2']

df_Calidad2 = df_Calidad2.loc[:, ~df_Calidad2.columns.str.startswith('Unnamed')]
df_Calidad2.columns = df_Calidad2.columns.str.strip()
df_Calidad2 = df_Calidad2.drop(columns=['FECHA','VERIFICACIÓN FISICA CAFÉ TOSTADO', 'LIBERACIÓN DE LOTE', 'RESPONSABLE'])
df_Calidad2 = df_Calidad2.drop(index=[0,1])
df_Calidad2 = df_Calidad2.iloc[:-17]

display(df_Calidad2.head())
display(df_Calidad2.tail())

,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
2,01-300822,Madre Laura,32.0,"10,5",14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
3,01-131022,Madre Laura,79.8,10.4,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
4,01-181022,Madre Laura,38.0,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
5,05-181022,Doña Dolly,43.0,10,14,"Chocolate dulce,fresas,miel,cuerpo cremoso,aci...",85
6,01-291022,Madre Laura,272.0,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84


,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
24,01-290323,Madre Laura,65.00,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
25,01-100423,Madre Laura,120.00,10.8,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
26,01-240423,Madre Laura,101.45,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
27,01-040523,Madre Laura,40.00,10.8,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
28,01-150623,Madre Laura,10.00,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
